In [100]:
import kaggle
import zipfile
import pandas as pd

In [101]:
# downloading a dataset
!kaggle datasets download -d rashikrahmanpritom/data-science-job-posting-on-glassdoor

Dataset URL: https://www.kaggle.com/datasets/rashikrahmanpritom/data-science-job-posting-on-glassdoor
License(s): CC0-1.0




  0%|          | 0.00/1.53M [00:00<?, ?B/s]
 65%|██████▌   | 1.00M/1.53M [00:00<00:00, 1.81MB/s]
100%|██████████| 1.53M/1.53M [00:00<00:00, 2.41MB/s]


In [102]:
# extracting a zip file
zipfile_name = 'data-science-job-posting-on-glassdoor.zip'
with zipfile.ZipFile(zipfile_name, 'r') as file:
    file.extractall()

*note*: at this oint, we have two files:

- Uncleaned_DS_jobs.csv
- Cleaned_DS_Jobs.csv

let us **get rid of** Cleaned_DS_Jobs.csv now

In [103]:
import os

In [104]:
file_to_rm = 'Cleaned_DS_Jobs.csv'
os.remove(file_to_rm)
# changing a name of a file
os.rename('Uncleaned_DS_jobs.csv', 'job_posting.csv')

In [105]:
# reading csv file as pandas dataframe
job_posting = pd.read_csv('job_posting.csv')

In [106]:
# checking our dataframe
job_posting.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              672 non-null    int64  
 1   Job Title          672 non-null    object 
 2   Salary Estimate    672 non-null    object 
 3   Job Description    672 non-null    object 
 4   Rating             672 non-null    float64
 5   Company Name       672 non-null    object 
 6   Location           672 non-null    object 
 7   Headquarters       672 non-null    object 
 8   Size               672 non-null    object 
 9   Founded            672 non-null    int64  
 10  Type of ownership  672 non-null    object 
 11  Industry           672 non-null    object 
 12  Sector             672 non-null    object 
 13  Revenue            672 non-null    object 
 14  Competitors        672 non-null    object 
dtypes: float64(1), int64(2), object(12)
memory usage: 78.9+ KB


so, ``job_posting`` dataframe consists of:
- 672 rows
- 15 columns

In [107]:
# how our uncleaned dataset looks like
job_posting.head()

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Sr Data Scientist,$137K-$171K (Glassdoor est.),Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,1,Data Scientist,$137K-$171K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
2,2,Data Scientist,$137K-$171K (Glassdoor est.),Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1
3,3,Data Scientist,$137K-$171K (Glassdoor est.),JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,4,Data Scientist,$137K-$171K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


right off the bat, there are several changes we can do to make our data more presentable:
1. in *Salary Estimate* column: get rid of dollar signs and *(Glassdoor est.)*
2. in *Company Name* column: left slice everything before newline character appearance
3. in *Job Description* column: get rid of newline chars, maybe replace them with ' '(?)
4. in a whole dataframe: replace *-1* and *Unknown / Non-Applicable* values to *Unknown*, since both of those values that we want to replace basically mean the same thing
5. add the following columns based on *Salary Estimate*:
    - *Min salary*
    - *Max salary*
    - *Average salary*
6. add *Same state* column, that will represent whether or not a companys location and headquartes are located in the same state

#### Min, Max, Average Salaries

In [118]:
# rm "(Glassdoor est.)" 
job_posting['Salary Estimate'] = job_posting['Salary Estimate'].map(lambda x: x.split('(')[0])

# function that returns min and max salaries
def seperate_est(est):
    min_sal, max_sal = est.split('-')

    # removing '$' nad 'K' from a string, converting it to int and multiplying it by 1000
    min_sal = int(min_sal.strip('$').replace('K', '')) * 1000
    max_sal = int(max_sal.strip('$').replace('K', '')) * 1000

    return min_sal, max_sal

# assigning 'Min Salary' and 'Max Salary' columns
job_posting[['Min Salary', 'Max Salary']] = job_posting['Salary Estimate'].apply(lambda x: pd.Series(seperate_est(x)))

# assigning a 'Average Salary' column
job_posting['Average Salary'] = (job_posting['Min Salary'] + job_posting['Max Salary']) / 2

# reasigning a 'Salary Estimate' column
job_posting['Salary Estimate'] = job_posting['Min Salary'].astype(str) + '-' + job_posting['Max Salary'].astype(str)

0      137000000-171000000
1      137000000-171000000
2      137000000-171000000
3      137000000-171000000
4      137000000-171000000
              ...         
667    105000000-167000000
668    105000000-167000000
669    105000000-167000000
670    105000000-167000000
671    105000000-167000000
Name: Salary Estimate, Length: 672, dtype: object

Now, let us have a look at columns we worked on.

In [119]:
job_posting[['Min Salary', 'Max Salary', 'Salary Estimate', 'Average Salary']].head()

,Min Salary,Max Salary,Salary Estimate,Average Salary
0,137000000,171000000,137000000-171000000,154000000.0
1,137000000,171000000,137000000-171000000,154000000.0
2,137000000,171000000,137000000-171000000,154000000.0
3,137000000,171000000,137000000-171000000,154000000.0
4,137000000,171000000,137000000-171000000,154000000.0


### Removing '\n' in a *Company Name* column

In *Company Name* column are mixed values.

In [122]:
job_posting['Company Name'].tail()

667                TRANZACT\n3.6
668                         JKGT
669                   AccessHope
670    ChaTeck Incorporated\n5.0
671           1-800-Flowers\n2.7
Name: Company Name, dtype: object

It looks like in some cases, rows have a syntax consisting of 'company name', '\n' and 'company rating'.

Let us extract only the name of the company.

In [124]:
job_posting['Company Name'] = job_posting['Company Name'].map(lambda x: x.split('\n')[0])
job_posting['Company Name'].tail()

667                TRANZACT
668                    JKGT
669              AccessHope
670    ChaTeck Incorporated
671           1-800-Flowers
Name: Company Name, dtype: object

Now it is all clean!

### '\n' chars inside *Job Description* column 

In [130]:
job_posting['Job Description'].head()

0    Description\n\nThe Senior Data Scientist is re...
1    Secure our Nation, Ignite your Future\n\nJoin ...
2    Overview\n\n\nAnalysis Group is one of the lar...
3    JOB DESCRIPTION:\n\nDo you have a passion for ...
4    Data Scientist\nAffinity Solutions / Marketing...
Name: Job Description, dtype: object

We need to remove every newline character in a column. However, if we use for example just a `.replace()` method, as we did before, we can be left with:

- no space between words, if we choose to replace \n with ''
- a huge gap between words, if we replace \n with ' '

So, we will have to ensure that no matter how many \n are in a string, there has to be only one space in their place.

In [145]:
# importing regex library
import re

def remove_newline(val):
    # replacing enery newline char with ' '
    val = val.replace('\n', ' ')

    # replacing any sequence of spaces with a single space
    val = re.sub(r'\s+', ' ', val)

    # returning a value with removed any possible leading or trailing spaces
    return val.strip()

# reassigning a 'Job Description' column
job_posting['Job Description'] = job_posting['Job Description'].apply(remove_newline)

In [146]:
# viewing our recently modified column
job_posting['Job Description'].head()

0      Description The Senior Data Scientist is respo...
1      Secure our Nation, Ignite your Future Join the...
2      Overview Analysis Group is one of the largest ...
3      JOB DESCRIPTION: Do you have a passion for Dat...
4      Data Scientist Affinity Solutions / Marketing ...
                             ...                        
667    Summary We’re looking for a data scientist to ...
668    Job Description Become a thought leader within...
669    Join a thriving company that is changing the w...
670    100 Remote Opportunity As an AINLP Data Scient...
671    Description The Data Scientist will be part of...
Name: Job Description, Length: 672, dtype: object

Looks magnificent!